In [47]:

from osdatahub import FeaturesAPI, Extent, NGD
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import mapclassify as mc
from shapely.geometry import Point
import os
import numpy as np
from convertbng.util import convert_bng, convert_lonlat
from os_functions import OSparam_feature, OSparam_ngd
from folium.plugins import measure_control
from pyaurn import importMeta

key = os.environ.get('OS_API_KEY')
crs = "EPSG:27700"

## Location Setup

In [3]:
AURN_meta = importMeta()
AURN_meta_ongoing = AURN_meta[AURN_meta['end_date'] == 'ongoing']
AURN_meta_ongoing = AURN_meta_ongoing[AURN_meta_ongoing['parameter'] == 'NO']

latlon = AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0]
eastnorth = convert_bng(AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0])

/tmp/ipykernel_63042/2180788136.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latlon = AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0]
/tmp/ipykernel_63042/2180788136.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  eastnorth = convert_bng(AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0])


In [4]:
X = eastnorth[0]
Y = eastnorth[1]
radius = 50
clip = True
product = 'topographic_area'

## Line bearings to Buildings

In [4]:
BuildingData = OSparam_feature(X, Y, radius,product,key, clip)
BuildingGeoSeries = BuildingData['geometry'][BuildingData['Theme'] == 'Buildings']
BuildingGeoSeries = BuildingGeoSeries.to_crs(27700)
BuildingDistances = BuildingGeoSeries.distance(Point(X,Y))
LineDistances = BuildingGeoSeries.shortest_line(Point(X,Y))

In [5]:
import folium
from shapely.geometry import Point, LineString

# Define the line and start/end points
line = LineDistances.iloc[0]
end_point = Point(line.coords[0])
start_point = Point(line.coords[-1])
line_points = [p for p in line.coords]
lats = convert_lonlat(*zip(*line_points))[0]
lons = convert_lonlat(*zip(*line_points))[1]

points = []
for i in range(len(lats)):
    points.append([lons[i], lats[i]])

start_lon, start_lat = convert_lonlat(start_point.x, start_point.y)
end_lon,end_lat = convert_lonlat(end_point.x, end_point.y)

# Define the projection
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')
fwd_azimuth,back_azimuth,distance = geodesic.inv(start_lon[0], start_lat[0], end_lon[0], end_lat[0])

# Create a folium map centered on the line
m = folium.Map(location=[start_lat[0],start_lon[0]], zoom_start=19)

# # Add the line to the map in blue
folium.PolyLine(points, color='blue',popup=fwd_azimuth).add_to(m)

# # Add the start point to the map in red
folium.Marker(location=[start_lat[0],start_lon[0]], icon=folium.Icon(color='green')).add_to(m)

# # Add the end point to the map in green
folium.Marker(location=[end_lat[0], end_lon[0]], icon=folium.Icon(color='red')).add_to(m)

# # Display the map
m
# points

## NGD

In [5]:
ngd_collections = NGD.get_collections()

In [6]:
product = 'trn-rami-averageandindicativespeed-1'
extent = Extent.from_radius((X,Y), 50, "EPSG:27700")

In [7]:
collections = {}

for i in ngd_collections['collections']:
    collections.update({i['title']:i['id']})

recipe = ['Average And Indicative Speed',
'Pavement Link',
'Pavement',
'Road Link',]

In [8]:
import difflib

data_collections = [collections[difflib.get_close_matches(i, collections.keys())[0]] for i in recipe]
data_collections


['trn-rami-averageandindicativespeed-1',
 'trn-ntwk-pavementlink-1',
 'trn-ntwk-pavementlink-1',
 'trn-ntwk-roadlink-2']

In [50]:
buildings = OSparam_feature(X, Y, radius,'topographic_area',key, clip)
buildings = buildings[buildings['Theme'] == 'Buildings'].set_crs(27700)
averagespeed = OSparam_ngd(X, Y, radius,data_collections[0],key, clip=False)
pavement = OSparam_ngd(X, Y, radius,data_collections[1],key, clip=False)
road = OSparam_ngd(X, Y, radius,data_collections[2],key, clip=False)

In [16]:
buildings_headers = ['AbsHMin','AbsH2', 'AbsHMax', 'RelH2', 'RelHMax', 'BHAConf', 'Shape_Length','Shape_Area']
# averagespeed_headers = # averagespeed.columns
# pavement_headers = 
pavement.columns
#road_headers = # road.columns

Index(['geometry', 'osid', 'theme', 'changetype', 'name1_text', 'name2_text',
       'roadlinkid', 'description', 'versiondate', 'name1_language',
       'name2_language', 'routehierarchy', 'geometry_length_m',
       'alternatename1_text', 'alternatename2_text', 'capturespecification',
       'versionavailabletodate', 'alternatename1_language',
       'alternatename2_language', 'roadclassificationnumber',
       'versionavailablefromdate', 'presenceofpavement_source',
       'presenceofpavement_sideofroad', 'presenceofpavement_updatedate',
       'presenceofpavement_evidencedate', 'presenceofpavement_capturemethod',
       'presenceofpavement_averagewidth_m',
       'presenceofpavement_minimumwidth_m'],
      dtype='object')

In [57]:
m = folium.Map(location=[latlon[1], latlon[0]], zoom_start=18)
fg = folium.FeatureGroup(name="data")
fg.add_child(folium.Marker([latlon[1], latlon[0]]))
fg.add_child(folium.GeoJson(buildings,popup=folium.GeoJsonPopup(fields=buildings_headers)))
fg.add_child(folium.GeoJson(averagespeed.set_crs(4326)))
fg.add_child(folium.GeoJson(pavement.set_crs(4326)))
fg.add_child(folium.GeoJson(road.set_crs(4326)))

m.add_child(fg)

m



## Features

In [67]:
from osdatahub.FeaturesAPI.feature_products import PREMIUM_NEW,OPEN_NEW
PREMIUM_NEW,OPEN_NEW

({'topographic_area': Product(name='Topography_TopographicArea', geometry='MultiPolygon'),
  'topographic_point': Product(name='Topography_TopographicPoint', geometry='Point'),
  'topographic_line': Product(name='Topography_TopographicLine', geometry='MultiLineString'),
  'water_network_link': Product(name='WaterNetwork_WatercourseLink', geometry='MultiLineString'),
  'water_network_node': Product(name='WaterNetwork_HydroNode', geometry='Point'),
  'path_network_link': Product(name='DetailedPathNetwork_RouteLink', geometry='MultiLineString'),
  'path_network_node': Product(name='DetailedPathNetwork_RouteNode', geometry='Point'),
  'highways_connecting_link': Product(name='Highways_ConnectingLink', geometry='MultiLineString'),
  'highways_connecting_node': Product(name='Highways_ConnectingNode', geometry='Point'),
  'highways_ferry_link': Product(name='Highways_FerryLink', geometry='MultiLineString'),
  'highways_ferry_node': Product(name='Highways_FerryNode', geometry='Point'),
  'high

In [92]:
features = FeaturesAPI(key, 'Highways_RoadLink', extent)
results = features.query(limit=500)
if len(results['features']) == 0:
    print('No features found')
TA_gdf = gpd.GeoDataFrame.from_features(results['features'], crs=crs)
TA_gdf.columns

Index(['geometry', 'GmlID', 'OBJECTID', 'ID', 'Identifier',
       'InspireIDNamespace', 'InspireIDLocalID', 'Fictitious',
       'RoadClassification', 'RouteHierarchy', 'FormOfWay', 'TrunkRoad',
       'PrimaryRoute', 'RoadClassificationNumber', 'RoadName1',
       'RoadName1Lang', 'RoadName2', 'RoadName2Lang', 'AlternateName1',
       'AlternateName1Lang', 'AlternateName2', 'AlternateName2Lang',
       'OperationalState', 'Provenance', 'Directionality', 'Length',
       'LengthUOM', 'MatchStatus', 'AlternateIdentifier',
       'StartGradeSeparation', 'EndGradeSeparation', 'RoadStructure',
       'CycleFacility', 'CycleFacilityWholeLink', 'RoadWidthConfidenceLevel',
       'RoadWidthAverage', 'RoadWidthMinimum', 'NumberOfLanes1',
       'NumberOfLanes1Direction', 'NumberOfLanes1MinMax', 'NumberOfLanes2',
       'NumberOfLanes2Direction', 'NumberOfLanes2MinMax', 'NumberOfLanes3',
       'NumberOfLanes3Direction', 'NumberOfLanes3MinMax', 'NumberOfLanes4',
       'NumberOfLanes4Direction